## Getting Started with Ray Actors

We assume that you've already read about [Ray Tasks](Ray Tasks.ipynb) and now you're ready to learn about the other half of the equation, Ray Actors. Ray Actors extend the Ray API to classes. An actor is essentially a stateful worker that runs in its own Python process. This allows it to work around the Python GIL.

In [1]:
import ray

In [8]:
@ray.remote
class Counter(object):
    def __init__(self):
        self.value = 0

    def increment(self):
        self.value += 1
        return self.value
    
    def get_value(self):
        return self.value

Now that we've created our class and annotated it properly with `@ray.remote` we can now refer to it in our code.

In [6]:
ray.shutdown()

In [7]:
ray.init()

2020-04-09 15:57:59,481	INFO resource_spec.py:212 -- Starting Ray with 2.39 GiB memory available for workers and up to 1.21 GiB for objects. You can adjust these settings with ray.init(memory=<bytes>, object_store_memory=<bytes>).
2020-04-09 15:57:59,738	INFO services.py:1123 -- View the Ray dashboard at localhost:8265


{'node_ip_address': '192.168.1.181',
 'redis_address': '192.168.1.181:48108',
 'object_store_address': '/tmp/ray/session_2020-04-09_15-57-59_469633_20028/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2020-04-09_15-57-59_469633_20028/sockets/raylet',
 'webui_url': 'localhost:8265',
 'session_dir': '/tmp/ray/session_2020-04-09_15-57-59_469633_20028'}

In [9]:
a1 = Counter.remote()
a2 = Counter.remote()

## Interacting with Ray Actors

We can now interact with Ray Actors by calling them in a similar way to calling Ray tasks. To do so we simply specify the method and add `.remote()`. For instance,

In [10]:
a1.increment.remote()

ObjectID(6f53dca1f451ca9444ee453c010000c801000000)

In this case, we can fetch the value by calling the `get_value` method, along with `ray.get`.

In [11]:
ray.get(a1.get_value.remote())

1

Consequentially, method calls on different actors will execute in parallel while methods called on the same actor are executed serially. Individual actors will maintain their own state, just like a normal Python class.

In [12]:
for x in range(35):
    if x % 2 == 0:
        a1.increment.remote()
    a2.increment.remote()

In [14]:
print(ray.get(a1.get_value.remote()))
print(ray.get(a2.get_value.remote()))

19
35


## Conclusion

Actors are a super powerful primitive with which to build your application.